# Composite Function

In [48]:
h = 1e-4 # 스텝
get_derivative = lambda f, x: (f(x+h) - f(x-h)) / (2*h) # 중앙 차분

class Function1:

    def __init__(self):
        self.function = lambda x: x - 2
    
    def forward(self, x):
        self.x = x
        z = self.function(x)
        return z
    
    def backward(self, dy_dx):
        self.dx = get_derivative(f=self.function, x=self.x)
        self.dout = self.dx * dy_dx
        return self.dout

class Function2:
    
    def __init__(self):
        self.function = lambda z: 2 * z**2

    def forward(self, z):
        self.z = z
        y = self.function(z)
        return y
    
    def backward(self):
        self.dx = get_derivative(f=self.function, x=self.z)
        self.dout = self.dx
        return self.dout
        
class Function:
    
    def __init__(self):
        self.function_1 = Function1()
        self.function_2 = Function2()
    
    def forward(self, x):
        print(f"Input x = {x}")
        z = self.function_1.forward(x)
        print(f"z = {z}")
        y = self.function_2.forward(z)
        print(f"y = {y}")
        return y

    def backward(self):
        f2_dout = self.function_2.backward()
        f1_dout = self.function_1.backward(f2_dout)
        print(f"Function2 dx = {self.function_2.dx}")
        print(f"Function1 dx = {self.function_1.dx}")
        print(f"Chain Rule dx = {f1_dout}")
        return f1_dout

function = Function()
function.forward(x=5)
function.backward()

# 전방 차분 결과 값
# Function2 dx = 12.000200000024108
# Function1 dx = 0.9999999999976694
# Chain Rule dx = 12.000199999996141

Input x = 5
z = 3
y = 18
Function2 dx = 12.000000000025324
Function1 dx = 0.9999999999976694
Chain Rule dx = 11.999999999997357


11.999999999997357

# Unit Function

In [149]:
import numpy as np
class Function:

    def __init__(self, function, d_function):
        self.function = function
        self.d_function = d_function
    
    def forward(self, x):
        self.x = x
        z = self.function(x)
        return z
    
    def backward(self, dy_dx=None):
        self.dx = self.d_function(self.x)
        if dy_dx is not None:
            self.dout = self.dx * dy_dx # 먼저 X를 전치하고 출력 미분과 행렬곱 해주면 된다.
        else:
            self.dout = self.dx
        return self.dout

# Sigmoid

In [150]:
import numpy as np

h = 1e-4
get_derivative = lambda f, x: (f(x+h) - f(x-h)) / (2*h)

class Sigmoid:
    
    def __init__(self):
        self.f1 = { "f": lambda x: -1 * x }
        self.f2 = { "f": lambda x: np.exp(x) }
        self.f3 = { "f": lambda x: 1 + x }
        self.f4 = { "f": lambda x: 1 / x }
        
    def forward(self, x):
        z1 = self.f1["f"](x)
        self.f1["x"] = x
        z2 = self.f2["f"](z1)
        self.f2["x"] = z1
        z3 = self.f3["f"](z2)
        self.f3["x"] = z2
        y = self.f4["f"](z3)
        self.f4["x"] = z3
        return y
    
    def backward(self, dout, dw=None):
        self.f4["dx"] = get_derivative(f=self.f4["f"], x=self.f4["x"]) * dw if dw is not None else dout
        self.f3["dx"] = get_derivative(f=self.f3["f"], x=self.f3["x"]) * self.f4["dx"]
        self.f2["dx"] = get_derivative(f=self.f2["f"], x=self.f2["x"]) * self.f3["dx"]
        self.f1["dx"] = get_derivative(f=self.f1["f"], x=self.f1["x"]) * self.f2["dx"]
        return dout, self.f1["dx"]
    
sigmoid = Sigmoid()
forward = sigmoid.forward(np.array([3, 8]))
print(f"Sigmoid forward = {forward}") # 0.9525741268224334

backward = sigmoid.backward(1)
print(f"Sigmoid backward = {backward}") # 0.04517666021639175


Sigmoid forward = [0.95257413 0.99966465]
Sigmoid backward = (1, array([-0.04978707, -0.00033546]))


# Affine

In [147]:
import numpy as np

class Affine:
    
    def __init__(self, shape, lr):
        self.w = np.random.randn(*shape) # wait 은 앞에꺼, b 는 뒤에꺼
        self.b = np.random.randn()
        self.lr = lr
        
    def forward(self, x):
        self.x = x
        print(f"dot {self.x @ self.w}")
        return self.x @ self.w + self.b
    
    def backward(self, dout, dw):
        print(f"dw {dw.shape}")
        self.dout_w = np.transpose(self.x) @ dw
        print(f"self.dout_w = {self.dout_w.shape}")
        self.w -= self.lr * self.dout_w
        self.b -= self.lr * dout
        return dout, self.dout_w

# Binary Classification Entropy Loss Function

In [151]:
import numpy as np
import copy
from functools import reduce

class BCELossFunction:
    
    def __init__(self, y):
        
        # - [ ylog(yhat) + (1-y)log(1-yhat) ]
        self.y = y.reshape(-1, 1)
        f0 = Function(function=lambda x: np.log(x),
                        d_function=lambda x: 1 / x)
        f1 = Function(function=lambda y_pred: y_pred[0] * y_pred[1],
                        d_function=lambda y: y[1]) # 변하는 값인 예측값에 대해 미분
        f2 = Function(function=lambda x: (1 - x),
                        d_function=lambda x: np.full(x.shape, -1))
        f3 = Function(function=lambda a_b: a_b[0] + a_b[1],
                        d_function=lambda a_b: a_b[0] + a_b[1])
        f4 = Function(function=lambda x: -x,
                        d_function=lambda x: np.full(x.shape, -1))
        
        self.f_list = [
            copy.deepcopy(f0),
            copy.deepcopy(f1),
            copy.deepcopy(f2),
            copy.deepcopy(f0),
            copy.deepcopy(f2),
            copy.deepcopy(f1),
            copy.deepcopy(f3),
            copy.deepcopy(f4)
        ]
        
    def forward(self, pred):
        step_1 = self.f_list[0].forward(pred)
        step_2 = self.f_list[1].forward((self.y, step_1)) # y가 1일때
        step_3 = self.f_list[2].forward(pred)
        step_4 = self.f_list[3].forward(step_3)
        step_5 = self.f_list[4].forward(self.y)
        step_6 = self.f_list[5].forward((step_5, step_4)) # y가 0일때
        step_7 = self.f_list[6].forward((step_2, step_6))
        loss_value = self.f_list[7].forward(step_7)
        return loss_value
    
    def backward(self):
        dout = reduce(lambda acc, f: f.backward(acc), 
                        self.f_list[::-1], 
                        None)
        return dout, dout
        
bce_loss = BCELossFunction(np.array([1, 1, 1, 1]))
_0_001 = bce_loss.forward(np.array([0.001, 0.002, 0.003, 0.004]).reshape(-1, 1))
print(f"_0_9 = {_0_001}")
print(bce_loss.backward())

_0_9 = [[6.90775528]
 [6.2146081 ]
 [5.80914299]
 [5.52146092]]
[[47.7887462 ]
 [38.73750175]
 [33.89855864]
 [30.67034821]]


# Artificial Neuron

In [153]:
class ArtificialNeuron:
    
    def __init__(self, shape, lr):
        self.affine = Affine(shape=shape, lr=lr)
        self.activation = Sigmoid()
        
    def forward(self, x):
        z = self.affine.forward(x)
        a = self.activation.forward(z)
        return a
    
    def backward(self, dout, dw):
        dout, dw = self.activation.backward(dout, dw)
        return self.affine.backward(dout, dw)
    

# Layer

In [3]:
class Layer:
    
    def __init__(self, shape, lr):
        # 인공 뉴런 리스트
        self.neurons = ArtificialNeuron(shape=shape, lr=lr)
        
    def forward(self, x):
        return self.neurons.forward(x)
    
    def backward(self, dout, dw):
        return self.neurons.backward(dout, dw)

# XOR ANN

In [154]:
import numpy as np
from functools import reduce

class XORNetwork():
    
    def __init__(self, neuron_count_list, lr, criterion_value):
        self.lr = lr
        self.criterion_value = criterion_value
        self.neuron_count_list = neuron_count_list
        
    
    def fit(self, x, y):
        x = np.array(x)
        y = np.array(y)
        self._set_neuron(x.shape[1], y) # Layer, Neuron 초기화
        loss_value = None # 손실함수 값
        for _ in range(1_000): # 최대 1000 번 학습
            loss_value = self._forward(x) # 순전파
            print(f"loss_value = {loss_value}")
            if np.all(loss_value < self.criterion_value): # 학습 종료 기준 값
                break
            self._backward() # 역전파
            
    def _set_neuron(self, x_size, y):
        self.layers = [] # Layer
        front_size = x_size
        if self.neuron_count_list == 1: # Layer 가 1개일 경우 shape
            self.layers = [ Layer(shape=(front_size, 1), lr=self.lr) ]
        else:
            self.layers = []
            for count in self.neuron_count_list: # Layer 별 Affine Shape 지정
                self.layers.append(Layer(shape=(front_size, count), lr=self.lr))
                front_size = count
        self.layers.append(BCELossFunction(y))
            
    def _forward(self, x) -> float:
        return reduce(lambda out, layer: layer.forward(out), self.layers, x) # 출력값 저장하고 다음으로 계속 넘겨주기 마지막은 손실함수

    def _backward(self) -> float:
        reduce(lambda out, layer: layer.backward(dout=out[0], dw=out[1]) if out is not None else layer.backward(), self.layers[::-1], None)

xor_ann = XORNetwork(neuron_count_list=[2, 1], lr=0.01, criterion_value=0.01)
xor_ann.fit([[0, 0],[0, 1],[1, 0],[1, 1]], [0, 1, 1, 0])

dot [[ 0.          0.        ]
 [ 0.82271076 -0.14138542]
 [ 0.09969753 -1.2096325 ]
 [ 0.92240829 -1.35101792]]
dot [[0.6789993 ]
 [0.98674037]
 [0.70245987]
 [1.00854988]]
loss_value = [[2.92138768]
 [0.04099389]
 [0.05411393]
 [3.23570173]]


TypeError: ArtificialNeuron.backward() missing 1 required positional argument: 'dw'